In [51]:
!pip install pandas openpyxl
!pip install requests
import pandas as pd
import requests
import os
from datetime import datetime

In [50]:
def eliminar_columnas_excel(df):
    # Columnas que queremos conservar
    columnas_a_conservar = ['CODIGO', 'DESCRIPCION', 'MARCA', 'PRECIO']
    # Eliminar todas las columnas que no sean las especificadas
    df = df[columnas_a_conservar]
    
    return df

In [24]:
def limpiar_formatos(df):
    # Eliminar espacios al principio y al final de todos los nombres de columnas
    df.columns = df.columns.str.strip()
    
    # Eliminar filas vacías y actualizar el DataFrame
    df = df.dropna(how='all').reset_index(drop=True)
    
    return df

In [18]:
def cargar_dataframe_con_encabezado(file_path, min_columnas=1):
    """
    Carga un DataFrame desde un archivo CSV o Excel. Si es CSV, lo convierte a Excel y lo lee.
    Busca la primera fila que contenga suficientes datos válidos para establecerla como encabezado.
    
    :param file_path: Ruta al archivo (puede ser CSV o Excel).
    :param min_columnas: Número mínimo de columnas no nulas requeridas para considerar una fila como encabezado.
    :return: DataFrame procesado con encabezados.
    """
    excel_file_path = None  # Variable para almacenar la ruta del archivo Excel a utilizar en el Paso 2
    df = None  # Inicializar df

    try:
        # Comprobación del tipo de archivo y ejecución del Paso 1
        if file_path.lower().endswith('.csv'):
            # Si el archivo es CSV, realizar la lectura y conversión a Excel
            print("El archivo es CSV, leyéndolo y convirtiéndolo a Excel...")
        
            try:
                # Leer el archivo CSV
                df = pd.read_csv(file_path, sep=';', on_bad_lines='skip')
                print("Archivo CSV leído correctamente.")
            
                # Convertir el CSV a Excel y guardar
                excel_file_path = os.path.splitext(file_path)[0] + ".xlsx"  # Crear el nombre del archivo Excel basado en el CSV
                df.to_excel(excel_file_path, index=False)
                print(f"Archivo CSV convertido y guardado como Excel en: {excel_file_path}")
        
            except FileNotFoundError:
                print(f"Error: No se encontró el archivo CSV en la ruta especificada: {file_path}")
                return None
            except pd.errors.EmptyDataError:
                print("Error: El archivo CSV está vacío.")
                return None
            except pd.errors.ParserError as e:
                print(f"Error al analizar el CSV: {e}")
                return None
            except Exception as e:
                print(f"Error inesperado al leer o convertir el archivo CSV: {e}")
                return None

        elif file_path.lower().endswith('.xlsx') or file_path.lower().endswith('.xls'):
            # Si el archivo es un Excel, definir directamente la ruta para el Paso 2
            print("El archivo es un Excel, se procede al Paso 2 directamente.")
            excel_file_path = file_path

        else:
            print("Error: El archivo no es ni CSV ni Excel. Asegúrate de que el archivo sea del tipo correcto.")
            return None

        # Paso 2: Realizar las operaciones correspondientes con el archivo Excel
        if excel_file_path:
            try:
                print(f"Paso 2: Leyendo el archivo Excel desde: {excel_file_path}")
            
                # Leer el archivo Excel creado o directamente el archivo proporcionado
                df = pd.read_excel(excel_file_path, header=None)  # Leer sin encabezado
                
                # Buscar la primera fila que contenga suficientes datos válidos
                for i in range(len(df)):
                    # Contar cuántas columnas no nulas hay en la fila actual
                    if df.iloc[i].notnull().sum() >= min_columnas:
                        # La fila i tiene suficientes datos válidos
                        df.columns = df.iloc[i]  # Establecer encabezados
                        df = df[i + 1:]  # Eliminar filas hasta la fila del encabezado
                        break
    
                # Reiniciar el índice y limpiar el DataFrame
                df.reset_index(drop=True, inplace=True)
                return df  # Retornar el DataFrame procesado
            
            except FileNotFoundError:
                print(f"Error: No se encontró el archivo Excel en la ruta especificada: {excel_file_path}")
                return None
            except pd.errors.EmptyDataError:
                print("Error: El archivo Excel está vacío.")
                return None
            except Exception as e:
                print(f"Error inesperado al leer el archivo Excel: {e}")
                return None
        else:
            print("No se encontró un archivo Excel válido para el Paso 2.")
            return None

    except Exception as general_error:
        print(f"Error general en la ejecución del script: {general_error}")
        return None


In [25]:
def limitar_descripcion(df):
    # Limitar la longitud de la columna DESCRIPCION a 100 caracteres
    df['DESCRIPCION'] = df['DESCRIPCION'].str.slice(0, 100)
    
    return df

In [52]:
def subir_archivo_a_api(file_path):
    """
    Sube un archivo a la API mediante una solicitud POST.
    
    :param file_path: Ruta al archivo a subir.
    :return: Respuesta de la API.
    """
    url = "https://desafio.somosait.com/api/upload/"
    try:
        with open(file_path, 'rb') as file:
            files = {'file': file}
            response = requests.post(url, files=files)

            if response.status_code == 200:
                print(f"Archivo {file_path} subido exitosamente.")
                return response.json()  # Retorna la respuesta en formato JSON
            else:
                print(f"Error al subir el archivo {file_path}: {response.status_code} - {response.text}")
                return None

    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta especificada: {file_path}")
        return None
    except Exception as e:
        print(f"Error inesperado al subir el archivo: {e}")
        return None

In [64]:
def guardar_dataframe(nombre_proveedor, df):
    """
    Guarda el DataFrame en un archivo Excel con un nombre dinámico basado en el proveedor y la fecha actual.

    :param nombre_proveedor: Nombre del proveedor que se usará en el nombre del archivo.
    :param df: DataFrame que se desea guardar.
    :return: Ruta completa del archivo guardado.
    """
    # Obtener la fecha actual en el formato deseado, por ejemplo: 'YYYY-MM-DD'
    fecha_hoy = datetime.today().strftime('%Y-%m-%d')

    # Crear el nombre del archivo concatenando nombre del proveedor y fecha
    nombre_archivo = f"{nombre_proveedor}_{fecha_hoy}.xlsx"

    # Obtener la ruta del directorio actual
    ruta_directorio = os.getcwd()  # Obtiene el directorio actual

    # Crear la ruta completa
    ruta_completa = os.path.join(ruta_directorio, nombre_archivo)

    # Guardar el archivo con el nombre dinámico
    df.to_excel(ruta_completa, index=False)

    # Imprimir la ruta completa del archivo guardado
    print(f"Archivo guardado como: {nombre_archivo}")
    print(f"Archivo guardado en: {ruta_completa}")  # Cambiar file_path a ruta_completa

    # Devolver la ruta completa
    return ruta_completa  # Retorna la ruta completa del archivo

In [73]:
def listas_autofix(file_path):
    sheets = pd.read_excel(file_path, sheet_name=None)  # 'sheet_name=None' carga todas las hojas
    
    # Lista para almacenar los dataframes procesados
    all_data = []
    
    # Iterar sobre cada hoja (marca)
    for marca, df in sheets.items():
        # Agregar la columna 'MARCA' con el nombre de la hoja
        df['MARCA'] = marca
        # Agregar el dataframe con la columna MARCA a la lista
        all_data.append(df)
    
    # Concatenar todos los dataframes en uno solo
    final_df = pd.concat(all_data, ignore_index=False)
    
    #Eliminar espacios al principio de las columnas y filas vacias
    final_df = limpiar_formatos(final_df)
    
    #Cambiar nombre de columna 
    final_df = final_df.rename(columns={'DESCR': 'DESCRIPCION'})
    
    #eliminar columnas
    final_df = eliminar_columnas_excel(final_df)
    
    #limitar caracteres de la columna DESCRIPCION
    fina_df = limitar_descripcion(final_df)
    
    #Corregir formato de PRECIO para que los decimales sean puntos
    final_df = final_df.copy() 
    final_df.loc[:, 'PRECIO'] = final_df['PRECIO'].astype(str).str.replace(',', '.')
    final_df['PRECIO'] = pd.to_numeric(final_df['PRECIO'], errors='coerce')
    
   # Guardar archivo y obtener la ruta
    excel_file_path = guardar_dataframe("AUTOFIX", final_df)

    # Subir el archivo a la API
    if excel_file_path:
        estado_api = subir_archivo_a_api(excel_file_path)
        print(estado_api)  # Imprime el estado de la API

    return final_df

    

In [40]:
# Ejecutar la función y almacenar el DataFrame final
df_final = listas_autofix(r"C:\Users\keidd\Downloads\AutoFix Repuestos.xlsx")

# Mostrar las primeras filas del DataFrame final
print("Vista previa del archivo final:")
print(df_final.head(5))

Archivo guardado como: AUTOFIX_2024-10-03.xlsx
Vista previa del archivo final:
                 CODIGO                                        DESCRIPCION  \
0  101995E                              KIT LARA FILTROS CRONOS 1.8 E TORQ   
1  101996E                   KIT LARA DISTRIBUCION C/ BOMBA AGUA  1.4 FIRE   
2  101999E               KIT LARA DISTRIBUCION PALIO/UNO C/BBA AGUA 1.4...   
3              3523540R             SOPORTE SUSPENSION DELANTERA       405   
4  412                       MANCHON CAMBIO T/ORIG. GACRI-020 FIAT 600 E.R   

  MARCA       PRECIO  
0    FI   90197.0400  
1    FI  133027.1232  
2    FI  136493.2800  
3    FI   34617.1300  
4    FI   22228.3100  


In [72]:
def listas_autorepuestos(file_path):
    # Cargar el archivo de Excel con todas las hojas
    final_df = cargar_dataframe_con_encabezado(file_path, min_columnas=4) 
   
    #Limpiar espacios, filas vacias
    final_df = limpiar_formatos(final_df)
    
    #Cambiar nombre de columnas 
    final_df = final_df.rename(columns={'PRECIO DE LISTA': 'PRECIO'})
    final_df = final_df.rename(columns={'CODIGO PROVEEDOR': 'CODIGO'})
    
    #eliminar columnas
    final_df = eliminar_columnas_excel(final_df)
    
    #limitar caracteres de la columna DESCRIPCION
    final_df = limitar_descripcion(final_df)
    
    # Guardar archivo y obtener la ruta
    excel_file_path = guardar_dataframe("AUTO REPUESTOS", final_df)

    # Subir el archivo a la API
    if excel_file_path:
        estado_api = subir_archivo_a_api(excel_file_path)
        print(estado_api)  # Imprime el estado de la API

    return final_df

In [43]:
# Ejecutar la función y almacenar el DataFrame final
autorepuestos_final = listas_autorepuestos(r"C:\Users\keidd\Downloads\AutoRepuestos Express.xlsx")

# Mostrar las primeras filas del DataFrame final
print("Vista previa del archivo final:")
print(autorepuestos_final.head(5))

El archivo es un Excel, se procede al Paso 2 directamente.
Paso 2: Leyendo el archivo Excel desde: C:\Users\keidd\Downloads\AutoRepuestos Express.xlsx
Archivo guardado como: AUTOREPUESTOS_2024-10-03.xlsx
Vista previa del archivo final:
10  CODIGO DESCRIPCION    MARCA   PRECIO
0   BR4068      BRAZOS  BUFFALO  6877.22
1   BR4089      BRAZOS  BUFFALO  5958.36
2   BR4090      BRAZOS  BUFFALO   4965.3
3   BR4149      BRAZOS  BUFFALO  5853.19
4   BR4153      BRAZOS  BUFFALO  7173.19


In [67]:
def listas_mundorepuestos(file_path):
    # Cargar el archivo con todas las hojas
    final_df = cargar_dataframe_con_encabezado(file_path, min_columnas=4) 
    
    # Limpiar espacios y eliminar filas vacías
    final_df = limpiar_formatos(final_df)

    # Verificar que las columnas necesarias existan antes de combinarlas
    if 'Descripcion' in final_df.columns and 'Rubro' in final_df.columns:
        # Crear la columna DESCRIPCION combinando "Descripcion" y "Rubro"
        # Asegúrate de que ambas columnas son cadenas de texto
        final_df['Descripcion'] = final_df['Descripcion'].fillna('').astype(str)
        final_df['Rubro'] = final_df['Rubro'].fillna('').astype(str)

        # Concatenar las columnas
        final_df['DESCRIPCION'] = final_df['Descripcion'] + ' ' + final_df['Rubro']
    else:
        raise ValueError("Las columnas 'Descripcion' y/o 'Rubro' no se encuentran en el DataFrame.")
    
    # Cambiar nombre de columnas
    final_df = final_df.rename(columns={
        'Importe': 'PRECIO',
        'Cod. Articulo': 'CODIGO',
        'Marca': 'MARCA'
    })
    
    # Eliminar columnas innecesarias
    final_df = eliminar_columnas_excel(final_df)

    # Limitar caracteres de la columna DESCRIPCION
    final_df = limitar_descripcion(final_df)

   # Guardar archivo y obtener la ruta
    excel_file_path = guardar_dataframe("MUNDO REPUESTOS", final_df)

    # Subir el archivo a la API
    if excel_file_path:
        estado_api = subir_archivo_a_api(excel_file_path)
        print(estado_api)  # Imprime el estado de la API

    return final_df

In [74]:
# Editar las ubicaciones del archivo para eL envio de las listas formateadas a la API.
listas_mundorepuestos(r"C:\Users\keidd\Downloads\AutoRepuestos Express Lista de Precios.csv")
listas_autorepuestos(r"C:\Users\keidd\Downloads\AutoRepuestos Express.xlsx")
listas_autofix(r"C:\Users\keidd\Downloads\AutoFix Repuestos.xlsx")



El archivo es CSV, leyéndolo y convirtiéndolo a Excel...
Archivo CSV leído correctamente.
Archivo CSV convertido y guardado como Excel en: C:\Users\keidd\Downloads\AutoRepuestos Express Lista de Precios.xlsx
Paso 2: Leyendo el archivo Excel desde: C:\Users\keidd\Downloads\AutoRepuestos Express Lista de Precios.xlsx
Archivo guardado como: MUNDO REPUESTOS_2024-10-04.xlsx
Archivo guardado en: C:\Users\keidd\automatización-web\MUNDO REPUESTOS_2024-10-04.xlsx
Archivo C:\Users\keidd\automatización-web\MUNDO REPUESTOS_2024-10-04.xlsx subido exitosamente.
{'link': 'https://docs.google.com/spreadsheets/d/1HVvKSTP-9SkPHhBvhO6Ze7aT4e-JBnrHHP1XiLIIe-U/edit?usp=sharing'}
El archivo es un Excel, se procede al Paso 2 directamente.
Paso 2: Leyendo el archivo Excel desde: C:\Users\keidd\Downloads\AutoRepuestos Express.xlsx
Archivo guardado como: AUTO REPUESTOS_2024-10-04.xlsx
Archivo guardado en: C:\Users\keidd\automatización-web\AUTO REPUESTOS_2024-10-04.xlsx
Archivo C:\Users\keidd\automatización-web\

,CODIGO,DESCRIPCION,MARCA,PRECIO
0,101995E,KIT LARA FILTROS CRONOS 1.8 E TORQ,FI,9.019704e+04
1,101996E,KIT LARA DISTRIBUCION C/ BOMBA AGUA 1.4 FIRE,FI,1.330271e+05
2,101999E,KIT LARA DISTRIBUCION PALIO/UNO C/BBA AGUA 1.4...,FI,1.364933e+05
3,3523540R,SOPORTE SUSPENSION DELANTERA 405,FI,3.461713e+04
4,412,MANCHON CAMBIO T/ORIG. GACRI-020 FIAT 600 E.R,FI,2.222831e+04
...,...,...,...,...
72707,1HM6981511-M,JUEGO PASTILLAS FRENO MOTRIO POLO / CADDY 00/,RN,3.495300e+04
72708,R1052734-O,VOLANTE MOTOR (VQ35DE) ORIG LAGUNA III,RN,3.691254e+06
72709,R19938148B,SOPORTE MOTOR TRAS. INF.GACRI FLUENCE...,RN,9.411136e+04
72710,R51303136N,"AMORTIGUADOR TRAS,NAKATA HG41234 LOGAN/SANDER...",RN,1.035835e+05
